In [1]:
import csv
import copy
import queue as Q

init_state = [[], [], [], [3, 1, 2]]
end_state = [[], [], [], [0, 0, 0]]

In [2]:
def read_file(filename):
    row_list = []
    with open(filename) as file:
        rows = csv.reader(file, delimiter=',')
        next(rows)
        for row in rows:
            row_list.append(row)
    return row_list

In [3]:
def init_offer_list(rows):
    offer_list = []
    for row in rows:
        row = row[1:]
        offer = [((i),eval(row[i])) for i in range(len(row)) if eval(row[i]) != 0]
        offer_list.append(offer)
    return offer_list

In [4]:
def init_cost_list(offer_list):
    cost_list = []
    for offer in offer_list:
        cost_list.append(offer[0][1])
    return cost_list

In [5]:
rows = read_file('ExpectedSalariesOfCandidates.csv')
offer_list = init_offer_list(rows)
cost_list = init_cost_list(offer_list)
print(offer_list)

[[(0, 50)], [(1, 30)], [(0, 85)], [(2, 55)], [(2, 45)], [(2, 30)], [(0, 80)], [(1, 55)], [(0, 30)], [(0, 35)], [(1, 70)], [(1, 90)], [(2, 20)], [(0, 90)], [(1, 55)], [(2, 35)], [(1, 35)], [(2, 45)], [(0, 40)], [(1, 80)]]


In [6]:
def bfs_v2(initial_state, goal_state, offer_list):
    explored = []
    queue = [initial_state]
    idx = 0
    visited = [initial_state]
    
    while queue:
        current = queue.pop(0)
        explored.append(current)
        
        if current[3] == goal_state[3]:
            return visited
        
        department_idx = offer_list[idx][0][0]
        
        rejected = copy.deepcopy(current)
        accepted = copy.deepcopy(current)
        if accepted[3][department_idx] > 0:
            accepted[department_idx].append(idx)
            accepted[3][department_idx] = accepted[3][department_idx]-1
        
        if accepted[3] == rejected[3]:
            queue.append(accepted)
        
        children = [accepted, rejected]

        for child in children:
            if child not in visited:
                queue.append(child)
                visited.append(child)
                
        idx = idx + 1
                
    return explored

In [7]:
bfs = bfs_v2(init_state, end_state, offer_list)
print(bfs)

[[[], [], [], [3, 1, 2]], [[0], [], [], [2, 1, 2]], [[0], [1], [], [2, 0, 2]], [[0, 2], [1], [], [1, 0, 2]], [[0, 2], [1], [3], [1, 0, 1]], [[0, 2], [1], [3, 4], [1, 0, 0]], [[0, 2, 6], [1], [3, 4], [0, 0, 0]]]


In [8]:
def dfs_v2(initial_state, goal_state, offer_list):
    visited = []
    stack = [initial_state]
    idx = 0
    
    while stack != []:
        current = stack.pop()
        
        if current not in visited:
            visited.append(current)
            
        if current[3] == goal_state[3]:
            return visited
        
        department_idx = offer_list[idx][0][0]
        
        accepted = copy.deepcopy(current)
        if accepted[3][department_idx] > 0:
            accepted[department_idx].append(idx)
            accepted[3][department_idx] = accepted[3][department_idx]-1
        
        if accepted[3] == current[3]:
            stack.append(accepted)
            
        children = [accepted, current]

        for child in reversed(children):
            if child not in visited:
                stack.append(child)
        
        idx = idx + 1
                
    return visited

In [9]:
dfs = dfs_v2(init_state, end_state, offer_list)
print(dfs)

[[[], [], [], [3, 1, 2]], [[0], [], [], [2, 1, 2]], [[0], [1], [], [2, 0, 2]], [[0, 2], [1], [], [1, 0, 2]], [[0, 2], [1], [3], [1, 0, 1]], [[0, 2], [1], [3, 4], [1, 0, 0]], [[0, 2, 6], [1], [3, 4], [0, 0, 0]]]


In [10]:
init_state_ucs = [[], [], [], [3, 1, 2], 0]
end_state_ucs = [[], [], [], [0, 0, 0], 0]

In [11]:
def ucs_v2(initial_state, goal_state, offer_list):
    visited = list()
    queue = Q.PriorityQueue()
    queue.put((0, initial_state, 0))
    
    while queue:
        print(queue.queue)
        cost, current, idx = queue.get()
        
        department_idx = offer_list[idx][0][0]
        print('department: ' + str(department_idx))
        
        visited.append(current)
        
        print('cost: ' + str(cost))
        print('visited: ' + str(visited))
        
        accepted = copy.deepcopy(current)
        rejected = copy.deepcopy(current)
        
        possible_costs = [-offer_list[idx][0][1], 0]
        
        if accepted[3][department_idx] > 0:
            accepted[department_idx].append(idx)
            accepted[3][department_idx] = accepted[3][department_idx]-1
            accepted[4] = accepted[4] + 1
            idx = idx + 1
            
        rejected[4] = rejected[4] + 1

        children = [accepted, rejected]
        
        for possible_cost, child in zip(possible_costs, children):
            if child not in visited:
                total_cost = cost + int(possible_cost)
                queue.put((total_cost, child, idx))
        

    return visited

In [12]:
ucs_v2(init_state_ucs, end_state_ucs, offer_list[:6])

[(0, [[], [], [], [3, 1, 2], 0], 0)]
department: 0
cost: 0
visited: [[[], [], [], [3, 1, 2], 0]]
[(-50, [[0], [], [], [2, 1, 2], 1], 1), (0, [[], [], [], [3, 1, 2], 1], 1)]
department: 1
cost: -50
visited: [[[], [], [], [3, 1, 2], 0], [[0], [], [], [2, 1, 2], 1]]
[(-80, [[0], [1], [], [2, 0, 2], 2], 2), (0, [[], [], [], [3, 1, 2], 1], 1), (-50, [[0], [], [], [2, 1, 2], 2], 2)]
department: 0
cost: -80
visited: [[[], [], [], [3, 1, 2], 0], [[0], [], [], [2, 1, 2], 1], [[0], [1], [], [2, 0, 2], 2]]
[(-165, [[0, 2], [1], [], [1, 0, 2], 3], 3), (-80, [[0], [1], [], [2, 0, 2], 3], 3), (-50, [[0], [], [], [2, 1, 2], 2], 2), (0, [[], [], [], [3, 1, 2], 1], 1)]
department: 2
cost: -165
visited: [[[], [], [], [3, 1, 2], 0], [[0], [], [], [2, 1, 2], 1], [[0], [1], [], [2, 0, 2], 2], [[0, 2], [1], [], [1, 0, 2], 3]]
[(-220, [[0, 2], [1], [3], [1, 0, 1], 4], 4), (-165, [[0, 2], [1], [], [1, 0, 2], 4], 4), (-50, [[0], [], [], [2, 1, 2], 2], 2), (0, [[], [], [], [3, 1, 2], 1], 1), (-80, [[0], [1], []

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


KeyboardInterrupt: 

###### 